In [5]:
from transformers import BertTokenizer, BertForQuestionAnswering
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, TensorDataset
from transformers import AdamW
import pandas as pd

# Example data
data = pd.read_json('assets//data.json')

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

encoded_data = []
max_len = 0

for index, row in data.iterrows():
    inputs = tokenizer(row['question'], row['context'], return_tensors='pt', truncation=True)
    start_positions = tokenizer(row['answer'], return_tensors='pt')['input_ids']

    encoded_data.append({
        'input_ids': inputs['input_ids'].squeeze(),
        'attention_mask': inputs['attention_mask'].squeeze(),
        'start_positions': start_positions.squeeze()
    })

    max_len = max(max_len, inputs['input_ids'].size(1))

# Pad sequences
padded_input_ids = pad_sequence([item['input_ids'] for item in encoded_data], batch_first=True, padding_value=tokenizer.pad_token_id)
padded_attention_mask = pad_sequence([item['attention_mask'] for item in encoded_data], batch_first=True, padding_value=0)
padded_start_positions = pad_sequence([item['start_positions'] for item in encoded_data], batch_first=True, padding_value=tokenizer.pad_token_id)

# Convert data to PyTorch tensors
input_ids = torch.tensor(padded_input_ids)
attention_mask = torch.tensor(padded_attention_mask)
start_positions = torch.tensor(padded_start_positions)

dataset = TensorDataset(input_ids, attention_mask, start_positions)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Fine-tuning parameters
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training loop
for epoch in range(3):  # Adjust as needed
    for batch in dataloader:
        optimizer.zero_grad()

        # Ensure that labels are provided for calculating the loss
        labels = batch[2]
        labels = labels.to(device)  # Ensure labels are on the same device as the model

        # Model forward pass with labels
        outputs = model(input_ids=batch[0].to(device), attention_mask=batch[1].to(device), start_positions=labels)
        loss = outputs.loss

        # Backward pass and optimization
        loss.backward()
        optimizer.step()



Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_11211/3949389849.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(padded_input_ids)
/tmp/ipykernel_11211/3949389849.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(padded_attention_mask)
/tmp/ipykernel_11211/3949389849.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or

NameError: name 'device' is not defined

In [ ]:
model.save_pretrained('fine_tuned_model')
tokenizer.save_pretrained('fine_tuned_model')